<h1>Introduction to HMM</h1>

This is a short guide through to hidden Markov Models. While HMM is widely used, the concepts for this workbook have been adopted by using the following resources:
<ul>
    <li><a href="https://web.ece.ucsb.edu/Faculty/Rabiner/ece259/Reprints/tutorial%20on%20hmm%20and%20applications.pdf">HMM Tutorial</a></li>
    <li><a href ="https://medium.com/@kangeugine/hidden-markov-model-7681c22f5b9"> An easy example</a></li>
    <li><a href="https://hmmlearn.readthedocs.io/en/latest/api.html#hmmlearn.hmm.GaussianHMM">hmmlearn Python API</a></li>
    <li><a href="https://www.adeveloperdiary.com/data-science/machine-learning/derivation-and-implementation-of-baum-welch-algorithm-for-hidden-markov-model/">Derivation of HMM for coding</a></li>
</ul>
    
The objective of this tutorial is to help in understanding of HMM and how to code it. It is not the objective to build an API or to code efficiently (but rather a lot for loops for better understanding of the underpinnings of the algorithm).

In HMMs, we observe some outcome variables (<b>$O_1,O_2...,O_T$</b>) which are driven from latent (hidden) variables. The probability of getting that outcome from one of the hidden state is called emission probability (and is denoted by <b>B</b>). In each time step, the latent state may change. However, this change depends only on the previous hidden state (hence it is called Markov model). These states transition with probability given by <b>A</b>. The model starts from an initial state. The initial state is denoted by <b>$\pi$</b>. 

The input parameters include - how many hidden states we would like to have in our model (denoted by <b>N</b>). Another parameter is the number of possible outcomes (in a discrete setting) and it is denoted by <b>M</b>. Thus a HMM model is denoted by the tuple $\lambda = (A,B,\pi)$. The notations are explained in the Figure below.

<img src="HMM.png" alt="HMM" width="628" height="628">

<h2>Three HMM problems</h2>

HMM are used to model three types of problems:
<ol>
    <li>Evaluation problem: If we know the model <b>(A,B,$\pi$)</b>, what is the probability of observing a given sequence?</li>
    <li>Decoding problem: If we know the model <b>(A,B,$\pi$)</b>, what is the best sequence of the hidden states that explain the sequence of the observations?</li>
    <li>Learning problem: How to estimate the value of <b>(A,B,$\pi$)</b> if we observe a given sequence of observations (or what model led to the generation of the given sequence). This is supervised HMM model.</li>
</ol>

In [207]:
# Consider a problem from https://towardsdatascience.com/introduction-to-hidden-markov-models-cd2c93e6b781
# outcome: hot (0) or cold (1)
# hidden states : snow, rain, sunshine
np.random.seed(42)
pi = np.array([0,0.2,0.8])
A  = np.array([[0.3,0.3,0.4],[0.1,0.45,0.45],[0.2,0.3,0.5]])
B  = np.array([[1,0],[0.8,0.2],[0.3,0.7]])
M  = 2 #(hot or cold)
N  = 3 # snow, rain, sunshine
T  = 20

# simulate the walk based on transition probability
# since this is generated based on actual probabilities, this state should have high probability of being observed
s = np.random.choice(3,1,p=pi)[0]
o = np.random.choice(2,1,p=B[s])[0]
S = [s]
O = [o]

for t in range(T-1):
    s = np.random.choice(3,1,p=A[s])[0]
    o = np.random.choice(2,1,p=B[s])[0]
    S.append(s)
    O.append(o)
print('observation  :',O)
print('hidden states:',S)

observation  : [1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1]
hidden states: [2, 2, 0, 0, 2, 0, 2, 0, 1, 1, 2, 1, 1, 1, 2, 2, 0, 2, 1, 2]


<h2>Evaluation problem: Forward algorithm and Backward algorithm</h2>

In this problem, we evaluate the probability of the observation sequence being observed in real life (if we already know the HMM). We first use forward algorithm and then use backward algorithm to show that we get same probability using both algorithms.


<img src="Evaluation.png" alt="forward-backward algorithm" width="828" height="628">

<h3>forward algorithm</h3>

In [208]:
# define alpha as the probability of partially observing the sequence upto t with state qt at time t
# initialization
alpha      = np.zeros((N,T))
alpha[:,0] = B[:,O[0]]*pi.T

# recursion            (s2 is the next state and s1 is the previous state)
for t in range(1,T):
    for s2 in range(N):
        for s1 in range(N):
            alpha[s2,t] += alpha[s1,t-1]*A[s1,s2]*B[s2,O[t]]
            
# final probability
prob_of_observing = np.sum(alpha[:,-1])
prob_of_observing

8.287718892764278e-06

<h3>backward algorithm</h3>

In [209]:
# define beta as the probability of partially observing the sequence from t+1 with state qt at time t
# initialization
beta       = np.zeros((N,T))
beta[:,-1] = 1   # because sequence is satisfied from T+1 onwards

# recursion            (s2 is the next state and s1 is the previous state)
for t in reversed(range(T-1)):
    for s1 in range(N):
        for s2 in range(N):
            beta[s1,t] += beta[s2,t+1]*A[s1,s2]*B[s2,O[t+1]]
            
# final probability
prob_of_observing = np.sum(beta[:,0]*B[:,O[0]]*pi.T)
prob_of_observing

8.287718892764281e-06

<h2>Decoding problem</h2>

In the decoding problem, we aim to find the best sequence for hidden states that led to the generation of the observation sequence as observed. The outline is shown below as we code for viterbi algorithm next. It is very similar to forward algorithm, just that in place of sum, we find the maximum.

<img src="Decoding.png" alt="viterbi algorithm" width="828" height="628">

<h3>Viterbi algorithm</h3>

In [219]:
# in viterbi algorithm, we still know the HMM model (A,B,pi), we just dont know the hidden state sequence
# the objective is to find the hidden state sequence to make inference. 

delta      = np.zeros((N,T))
delta[:,0] = B[:,O[0]]*pi.T    
psi        = np.array([0]*T)   # keeps a track of the best sequence

# recursion            (s2 is the next state and s1 is the previous state)
for t in range(1,T):
    for s2 in range(N):
        vals = [0]*N
        for s1 in range(N):
            vals[s1] = delta[s1,t-1]*A[s1,s2]*B[s2,O[t]]
        psi[t-1]    = np.argmax(vals)
        delta[s2,t] = vals[psi[t-1]]

# last state is directly based on the last observation
psi[t] = np.argmax(B[:,O[T-1]])
# optimal hidden state sequence
print('original hidden state:    ', S)
print('hidden state from viterbi:',psi)

original hidden state:     [2 2 0 0 2 0 2 0 1 1 2 1 1 1 2 2 0 2 1 2]
hidden state from viterbi: [2 2 1 1 2 1 1 1 1 1 1 1 1 1 1 1 1 2 1 2]


<h2>Learning problem: EM algorithm</h2>

Learning problem is considered to be the tough problem of finding the actual HMM parameters (A,B, $\pi$). This problem is also called supervised learning problem as parameters are obtained by looking into the outcomes. The values of A,B and $\pi$ can be obtained using gradient based optimization methods. We can also use other optimization routines likes bayesian optimization to find the parameters. Optimization methods are based on finding the error between true observation and observation prediction from the optimization routing. EM algorithm is more statistical flavor as it is based on maximizing the maximum likelihood. 

Here, we will use EM algorithm for finding the HMM parameters. HMM uses <b>Baum-Welch</b> algorithm which is a special case of Expectation maximization algorithm.


<img src="Learning.png" alt="baum welch algorithm" width="828" height="628">

In [184]:
# initializing A and B
# assuming we know the information on actual value of pi 
# if there are multiple sequences, it will just be the average of the number of times a state exist in the first step

A = np.ones((N,N))/N
B = np.ones((N,M))/M


# starting with the iteration
# we can create a function for forward and backward algorithm but for the sake of ease of following through, just copy pasted 
# here. We will use the same code as before to calculate eta and gamma


for iter in range(10):
    # ------------- expectation step ------------------
    # use A and B to obtain gamma and eta
    
    # alpha (from forward algorithm)
    alpha      = np.zeros((N,T))
    alpha[:,0] = B[:,O[0]]*pi.T
    
    for t in range(1,T):
        for s2 in range(N):
            for s1 in range(N):
                alpha[s2,t] += alpha[s1,t-1]*A[s1,s2]*B[s2,O[t]]
    
    # beta (from backward algorithm)
    beta       = np.zeros((N,T))
    beta[:,-1] = 1   # because sequence is satisfied from T+1 onwards
    
    for t in reversed(range(T-1)):
        for s1 in range(N):
            for s2 in range(N):
                beta[s1,t] += beta[s2,t+1]*A[s1,s2]*B[s2,O[t+1]]
    
    # emission based on the known observation sequence
    emission = np.zeros((N,T))
    for t in range(T):
        emission[:,t] = B[:,O[t]]
        
    # gamma
    gamma = np.multiply(alpha,beta)
    gamma /= np.sum(gamma,axis=0)
    
    # eta
    eta   = np.zeros((N,N,T))
    for t in range(T):
        sum_t = 0
        for s1 in range(N):
            if t == T-1:
                eta[s1,:,t] = alpha[s1,t]*1*A[s1,:]*1
                sum_t       += eta[s1,s2,t]
            else:
                eta[s1,:,t] = alpha[s1,t]*beta[:,t+1]*A[s1,:]*emission[:,t+1]
                sum_t       += eta[s1,:,t]

        eta[:,:,t]/= np.sum(sum_t)
                
    # ------------- maximization step ------------------
    # use eta and gamma to obtain A and B

    A = np.sum(eta,axis=2)
    A/= np.sum(A,axis=0)
    
    B = np.zeros((N,M))
    for m in range(M):
        hh = np.array(O)==m
        hh = hh.reshape(1,T)  # counting as how many times a particular obersvation was made
        B[:,m:m+1] = np.sum(np.multiply(gamma,hh),axis=1).reshape(N,1)/np.sum(gamma,axis=1).reshape(N,1)
        
    #print(np.sum(A,axis=0),np.sum(A,axis=1),np.sum(A))

In [185]:
A,B

(array([[0.23532977, 0.26146183, 0.35918148],
        [0.2674642 , 0.28013996, 0.31541464],
        [0.49720603, 0.45839821, 0.32540388]]),
 array([[0.87474069, 0.12525931],
        [0.765226  , 0.234774  ],
        [0.30732432, 0.69267568]]))

In [186]:
# the number of ietrations can be control for to avoid over - fitting. 
# NoTE that we only consider one sequence of observations. Generally we have multiple sequences
# We aim to learn the HMM model based on those sequences (by following the same procedure)

<h2>Using hmmlearn API</h2>

Now that we have learned how to code for HMM and it can help us in understanding the model, we can use existing libraries to build an HMM model. We can use the same problem to see how to use the hmmlearn library.

In [202]:
from hmmlearn import hmm
import math

In [213]:
np.random.seed(42)

pi = np.array([0,0.2,0.8])
A  = np.array([[0.3,0.3,0.4],[0.1,0.45,0.45],[0.2,0.3,0.5]])
B  = np.array([[1,0],[0.8,0.2],[0.3,0.7]])
M  = 2 #(hot or cold)
N  = 3 # snow, rain, sunshine
T  = 20

# simulate the walk based on transition probability
# since this is generated based on actual probabilities, this state should have high probability of being observed
s = np.random.choice(3,1,p=pi)[0]
o = np.random.choice(2,1,p=B[s])[0]
S = [s]
O = [o]

for t in range(T-1):
    s = np.random.choice(3,1,p=A[s])[0]
    o = np.random.choice(2,1,p=B[s])[0]
    S.append(s)
    O.append(o)
    
O = np.array(O)
S = np.array(S)
print('observation  :',O)
print('hidden states:',S)

observation  : [1 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1]
hidden states: [2 2 0 0 2 0 2 0 1 1 2 1 1 1 2 2 0 2 1 2]


<h3>Evaluation problem</h3>

In [205]:
model = hmm.MultinomialHMM(n_components=3)
model.startprob_ = pi
model.transmat_  = A
model.emissionprob_ = B

In [206]:
math.exp(model.score(O.reshape(1,-1)))
# note that it exactly matches the result in the forward and backward algorithms in Section 2.1 and 2.2

8.2877188927643e-06

<h3>Decoding problem</h3>

In [220]:
logprob, seq = model.decode(O.reshape(1,-1).transpose())
print(math.exp(logprob))
print(seq)
# note that the sequence is exactly same as the viterbi algorithm in Section 3.1

4.0128365182460964e-10
[2 2 1 1 2 1 1 1 1 1 1 1 1 1 1 1 1 2 1 2]


<h3>Learning problem</h3>

In [225]:
remodel = hmm.MultinomialHMM(n_components=3, n_iter=5)
remodel.fit(O.reshape(1,-1))

MultinomialHMM(algorithm='viterbi', init_params='ste', n_components=3, n_iter=5,
               params='ste',
               random_state=<mtrand.RandomState object at 0x000002046923BB40>,
               startprob_prior=1.0, tol=0.01, transmat_prior=1.0,
               verbose=False)

In [226]:
remodel.startprob_

array([2.17705391e-03, 9.97473874e-01, 3.49071843e-04])

In [227]:
remodel.transmat_

array([[0.38392689, 0.07199623, 0.54407688],
       [0.32899046, 0.30694324, 0.3640663 ],
       [0.38358656, 0.07685537, 0.53955807]])

In [228]:
remodel.emissionprob_

array([[0.82336472, 0.17663528],
       [0.11720271, 0.88279729],
       [0.8922925 , 0.1077075 ]])

There are different variants of HMM. we used an example that was multinomial (discrete outcomes). The outcomes can be different distributions (for example multinomial Gaussian) or single continuous outcome (like mixture models). All these are useful models to keep in mind when considering the model.